In [10]:
# Question 3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib

df = pd.read_csv('listings.csv')

def separator(df):
    '''
    Input: pandas dataframe with a column called "amenities"
    
    Output: A new pandas dataframe which only includes the amenities column
    
    This function checks the different amenities and returns a new dataframe with all different amenities:
    
    1. Cleaning the strings and split the amenities in different rows.
    2. Checking which amenities exist, which are available in the whole column.
    3. Creates a new dataframe with the amenities in the columns.
    '''
    
    ## Cleaning the strings and splitting each amenity into a different column
    df['amenities'] = df['amenities'].replace('{|}| |"', '', regex=True)
    df['amenities'] = df['amenities'].replace(',', ' ', regex=True)
    df_ame = pd.DataFrame(df['amenities'].str.split(expand=True))
    
    ## Checking which amenities exist and save them in a list "amenities"
    amenities = []
    for u in range(df_ame.shape[0]):
        for uu in range(df_ame.shape[1]):
            if (df_ame[uu][u] in amenities):
                True
            else:
                amenities += [df_ame[uu][u]]
    amenities = list(filter(None,amenities)) # delete None
    df_ame = df_ame.fillna(value=np.nan)
    
    ## Creates a new dataframe df_new with all amenities in a separate column (1 for given, 0 for not available)  
    df_new = pd.DataFrame(index=range(len(df_ame)), columns=amenities)
    for i in range(len(df_new)):
        for ii in range(df_ame.shape[1]):
            var = df_ame.iloc[i][ii]
            if (df_ame.iloc[i][ii] in amenities):
                df_new[var].iloc[i] = 1
            else:
                False
    df_new = df_new.fillna(0)
    df_new = df_new.astype(int)
    
    return df_new

def clean_data(df, flag):
    '''
    INPUT
    df - pandas dataframe 

    OUTPUT
    X - A matrix holding all of the variables you want to consider when predicting the response
    y - the corresponding response vector

    This function cleans df using the following steps to produce X and y:
    1. Optional: Use the separator function to split the amenities column if you are interested in these features
    2. Change all columns which contain "t" and "f" into 1 and 0
    3. Change the price columns to float variables
    4. Change the percent columns to float variables
    5. Drop all rows in which all rows have the same value
    6. Drop all the object columns. (Many of them are individual for each row and disturb the processing later)
    7. Drop rows with missing price values, create y
    8. Drop columns which have NAN in more than 75%
    9. Drop the columns, which include prices, additionally the "host_listings_count" and "host_total_listings_count"
        are dropped because these columns have few outliers and disturb the processing later. 
    10. For each numeric variable in X, fill all NAN of the column with the mean value of the column.
    '''

    # Split the amenities column into the different variables with the separator function
    # The amenities can be excluded by setting the flag to 0
    # The separator function needs some time to run
    if flag == 1:
        df_ame = separator(df)
        df = pd.concat([df,df_ame],axis=1)
        
    # Change the columns with "t" and "f" into 1 and 0
    df = df.replace('t',1).replace('f',0)

    # Change the dollar prices from string to float
    dollar_column = ['price','weekly_price','monthly_price']
    for dollar_count in dollar_column:
        df[dollar_count] = df[dollar_count].str.replace('$', '').str.replace(',', '').astype(float)

    # Change the columns with percent from string to float
    percent_column = df.select_dtypes(include=['object']).columns
    for percent_count in percent_column:
        try:
            df[percent_count] = df[percent_count].str.replace('%', '').astype(float)
        except:
            True

    # Drop all columns in which all rows have the same value
    nunique = df.apply(pd.Series.nunique)
    cols_to_drop = nunique[nunique == 1].index
    df = df.drop(cols_to_drop, axis=1)

    # Drop object columns
    cat_vars = df.select_dtypes(include=['object'])
    df = df.drop(cat_vars,axis=1)
    
    # Drop rows with missing price values
    if (df['price'].isnull().mean() != 0):
        df = df.dropna(subset=['price'], axis=0)
    y = df['price']

    # Drop columns with more than 75% zeros
    too_many_NAN = list(df.columns[df.isnull().mean() > 0.75])
    for count_2 in too_many_NAN:
        df = df.drop([count_2], axis=1)

    # Drop price columns
    df = df.drop(['price','weekly_price','monthly_price'], axis=1)
    df = df.drop(['host_listings_count','host_total_listings_count'],axis=1)
    
    # Fill numeric columns with the mean
    num_vars = df.select_dtypes(include=['float', 'int32']).columns
    for col in num_vars:
        df[col].fillna((df[col].mean()), inplace=True)    

    X = df
    return X, y

def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

# Say if amenities should be included or not
flag = 0 # 1 for yes, 0 for no

# Use the Clean_data function to create X and y
X, y = clean_data(df, flag)

# Split the data in a test and training set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state = 6)

# Create a LinearRegression model with normalized data
lm_model = LinearRegression(normalize=True)

# Fit our model with the training data
lm_model.fit(X_train,y_train)

# Predict our data using our test set
y_pred = lm_model.predict(X_test)

# Compare the predicted with the test data and return the r-value
rsquared_score = r2_score(y_test,y_pred)

# Use the function
coef_df = coef_weights(lm_model.coef_, X_train)

# A quick look at the top results and the r-squared-Value
print('The r-squared-score is: ' + str(rsquared_score))

# Plot the results in a bar plot
features = 20
ax = plt.bar(coef_df['est_int'][:features],coef_df['abs_coefs'][:features])
y_pos = range(len(coef_df['est_int'][:features]))
plt.xticks(y_pos, coef_df['est_int'][:features], rotation=90)
if flag == 0:
    plt.title('Important features on the airbnb price\n',fontsize = 20)
elif flag == 1:
    plt.title('Important features on the airbnb price (ammenities included)\n',fontsize = 20)
plt.ylabel('absolute coefficient')
plt.subplots_adjust(bottom=0.4)

Using matplotlib backend: Qt5Agg
The r-squared-score is: 0.5672529232782865
